# Assignment #3
## P556: Applied Machine Learning

More often than not, we will use a deep learning library (Tensorflow, Pytorch, or the wrapper known as Keras) to implement our models. However, the abstraction afforded by those libraries can make it hard to troubleshoot issues if we don't understand what is going on under the hood. In this assignment you will implement a fully-connected and a convolutional neural network from scratch. To simplify the implementation, we are asking you to implement static architectures, but you are free to support variable number of layers/neurons/activations/optimizers/etc. We recommend that you make use of private methods so you can easily troubleshoot small parts of your model as you develop them, instead of trying to figure out which parts are not working correctly after implementing everything. Also, keep in mind that there is code from your fully-connected neural network that can be re-used on the CNN. 

Problem #1.1 (40 points): Implement a fully-connected neural network from scratch. The neural network will have the following architecture:

- Input layer
- Dense hidden layer with 512 neurons, using relu as the activation function
- Dropout with a value of 0.2
- Dense hidden layer with 512 neurons, using relu as the activation function
- Dropout with a value of 0.2
- Output layer, using softmax as the activation function

The model will use categorical crossentropy as its loss function. 
We will optimize the gradient descent using RMSProp, with a learning rate of 0.001 and a rho value of 0.9.
We will evaluate the model using accuracy.

Why this architecture? We are trying to reproduce from scratch the following [example from the Keras documentation](https://keras.io/examples/mnist_mlp/). This means that you can compare your results by running the Keras code provided above to see if you are on the right track.

In [0]:
from sklearn.metrics import accuracy_score

import numpy as np
#np.random.seed(1)


class NeuralNetwork(object):    
    def __init__(self, epochs, learning_rate, rho):
        self.epochs=epochs
        self.learning_rate=learning_rate
        self.rho=rho
        self.weight1=0
        self.weight2=0
        self.weight3=0
        self.bias1=0
        self.bias2=0
        self.bias3=0


    
    def relu(self,hx):
        return (np.maximum(hx, 0))
    
    def relu_derivative(self,hx):
        hx[hx<=0]=0
        hx[hx>0]=1
        return(hx)
#        if hx>0:
#            return 1
#        else:
#            return 0
        
        
    def compute(self, h, theta, bias):
        output=np.dot(h, theta) + bias
        
#        print('output.shape ',output.shape,' output ',output)
        return output


    def rmsprop(self,sdw,dw):
        sdw=self.rho*sdw + (1-self.rho)*dw*dw
        return sdw
    
    
    def softmax(self,yhat):
      ymean=np.mean(yhat)
      ystd=np.std(yhat)
      yhat=(yhat-ymean)/ystd
      ex = np.exp(yhat)
      p  = ex/np.sum(ex,axis=0)
#        print(p[0])
      return p
        
#        ex = np.exp(yhat)
#        return (ex/np.sum(ex))


    def cross_entropy_loss(self,prob,y):
        
        m = y.shape[0]

        logprob=np.log(prob.clip(min=0.000000001))*-1
        product=logprob*y
        total_loss=np.sum(product)
        # total_loss=np.sum((y-prob)**2)
#        print('total_loss ',total_loss)

        zero_one=(prob == prob.max(axis=1)[:,None]).astype(int)
        accuracy=accuracy_score(y, zero_one,normalize=True)         
        
#        diff=(y == zero_one).sum(axis=1)
#        print('diff',diff)
#        accuracy=np.count_nonzero(diff == y.shape[1])
#        print('accuracy ',accuracy)       
        
        
#        loss = np.sum(log_likelihood) / m
#        return loss
    
    
    def cross_entropy_loss_derivative(self,yhat,y):
        diff=yhat-y
        return diff
        
    def fit(self,X,y):
        X_original=X
        y_original=y
        
        sdw3=sdw2=sdw1=0
        sdb3=sdb2=sdb1=0
        
        m=X.shape[0]
        
        w1=np.random.uniform(-1,1,size=(X.shape[1],512))*np.sqrt(1/X.shape[1])
        b1=np.zeros((1,512))
        
        w2=np.random.uniform(-1,1,size=(512,512))*np.sqrt(1/512)
        b2=np.zeros((1,512))
        
        w3=np.random.uniform(-1,1,size=(512,y.shape[1]))*np.sqrt(1/512)
        b3=np.zeros((1,y.shape[1]))
        
        dropout=0.2
        keep_probability=1-dropout

        for i in range(self.epochs):
            print('epoch ',i+1)

            sdw3=sdw2=sdw1=0            
            sdb3=sdb2=sdb1=0
            
            length=X_original.shape[0]
            step_size=5000
            iterations=np.ceil(length/step_size)
            iterations=int(iterations)
            
            for j in range(iterations):
                X=X_original[j*step_size:(j+1)*step_size]
                y=y_original[j*step_size:(j+1)*step_size]
                m=X.shape[0]

                o1=self.compute(X, w1, b1)
                r1=self.relu(o1)
                mask1=np.random.randn(r1.shape[0],r1.shape[1])<keep_probability
                d1=r1*mask1
                d1/=keep_probability
                
                
                o2=self.compute(d1, w2, b2)
                r2=self.relu(o2)
                mask2=np.random.randn(r2.shape[0],r2.shape[1])<keep_probability
                d2=r2*mask2
                d2/=keep_probability

                
                
                o3=self.compute(d2, w3, b3)
                s3=self.softmax(o3)
                
                l=self.cross_entropy_loss(s3,y)
                
    
                
                gw3=(y-s3)*self.cross_entropy_loss_derivative(o3,y)
                # gw3=self.cross_entropy_loss_derivative(o3,y)
                dw3=np.dot(d2.T,gw3)/m
                db3=np.sum(gw3,axis=0)
                
                
                gw2=np.dot(gw3,w3.T)*self.relu_derivative(d2)*mask2
                dw2=np.dot(d1.T,gw2)/m
                db2=np.sum(gw2,axis=0)
                
                
                gw1=np.dot(gw2,w2.T)*self.relu_derivative(d1)*mask1
                dw1=np.dot(X.T,gw1)/m
                db1=np.sum(gw1,axis=0)
                
    
                
                sdw3=self.rmsprop(sdw3,dw3)
                sdw2=self.rmsprop(sdw2,dw2)
                sdw1=self.rmsprop(sdw1,dw1)
                sdb3=self.rmsprop(sdb3,db3)
                sdb2=self.rmsprop(sdb2,db2)
                sdb1=self.rmsprop(sdb1,db1)
                
                
                w3=w3-self.learning_rate*dw3/np.sqrt(sdw3.clip(min=0.000001))
                w2=w2-self.learning_rate*dw2/np.sqrt(sdw2.clip(min=0.000001))
                w1=w1-self.learning_rate*dw1/np.sqrt(sdw1.clip(min=0.000001))
                
                b3=b3-self.learning_rate*db3/np.sqrt(sdb3.clip(min=0.000001))
                b2=b2-self.learning_rate*db2/np.sqrt(sdb2.clip(min=0.000001))
                b1=b1-self.learning_rate*db1/np.sqrt(sdb1.clip(min=0.000001))
                

                # w3=w3-self.learning_rate*dw3
                # w2=w2-self.learning_rate*dw2
                # w1=w1-self.learning_rate*dw1
                
                # b3=b3-self.learning_rate*db3
                # b2=b2-self.learning_rate*db2
                # b1=b1-self.learning_rate*db1

        self.weight1=w1
        self.weight2=w2
        self.weight3=w3

        self.bias1=b1
        self.bias2=b2
        self.bias3=b3

        return(w1,w2,w3,b1,b2,b3)
    

    def evaluate(self,X):

      output1=X@self.weight1+self.bias1
      relu1=self.relu(output1)

      output2=relu1@self.weight2+self.bias2
      relu2=self.relu(output2)

      output3=relu2@self.weight3+self.bias3
      soft3=self.softmax(output3)

      zero_one=(soft3 == soft3.max(axis=1)[:,None]).astype(int)
      return(zero_one)

    def predict(self,X,w1,w2,w3,b1,b2,b3):

      output1=X@w1+b1
      relu1=self.relu(output1)

      output2=relu1@w2+b2
      relu2=self.relu(output2)

      output3=relu2@w3+b3
      soft3=self.softmax(output3)

      zero_one=(soft3 == soft3.max(axis=1)[:,None]).astype(int)
      return(zero_one)



      # o1=self.compute(X, w1, b1)
      # r1=self.relu(o1)
      # mask1=np.random.randn(r1.shape[0],r1.shape[1])<keep_probability
      # d1=r1*mask1
      # # d1/=keep_probability
      
      
      # o2=self.compute(d1, w2, b2)
      # r2=self.relu(o2)
      # mask2=np.random.randn(r2.shape[0],r2.shape[1])<keep_probability
      # d2=r2*mask2
      # # d2/=keep_probability

      
      
      # o3=self.compute(d2, w3, b3)
      # s3=self.softmax(o3)

      pass

Problem #1.2 (10 points): Train your fully-connected neural network on the Fashion-MNIST dataset using 5-fold cross validation. Report accuracy on the folds, as well as on the test set.

In [3]:
# To simplify the usage of our dataset, we will be importing it from the Keras 
# library. Keras can be installed using pip: python -m pip install keras

# Original source for the dataset:
# https://github.com/zalandoresearch/fashion-mnist

# Reference to the Fashion-MNIST's Keras function: 
# https://keras.io/datasets/#fashion-mnist-database-of-fashion-articles
import keras
from sklearn.metrics import accuracy_score
from keras.datasets import fashion_mnist
from sklearn.model_selection import KFold
kf = KFold(n_splits=2)


# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




# cross validation starts

from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
c=kf.get_n_splits(x_train)
a=NeuralNetwork(10,0.001,0.9)
best_accuracy=0
cv=1
for train_index, test_index in kf.split(x_train):
    print('K Fold Cross Validation no.: ',cv)
    cv+=1
    
    x_tr=x_train[train_index]
    y_tr=y_train[train_index]
    
    (w1,w2,w3,b1,b2,b3)=a.fit(x_tr,y_tr)
    
    x_valid=x_train[test_index]
    y_valid=y_train[test_index]
    
    y_predicted=a.evaluate(x_valid)
    
    v_accuracy=accuracy_score(y_valid, y_predicted,normalize=True)
    
    print('validation accuracy ',v_accuracy)
    
    if v_accuracy>best_accuracy:
        (w1f,w2f,w3f,b1f,b2f,b3f)=(w1,w2,w3,b1,b2,b3)
        best_accuracy=v_accuracy
    
    y_training_predicted=a.evaluate(x_tr)    
    train_accuracy=accuracy_score(y_tr, y_training_predicted,normalize=True)
    
    print('training accuracy ',train_accuracy)
        
    
        
y_test_predicted=a.predict(x_test,w1f,w2f,w3f,b1f,b2f,b3f)
accuracy_test=accuracy_score(y_test, y_test_predicted,normalize=True)

#print('best weights and bias ',w1f,w2f,w3f,b1f,b2f,b3f)
print('\n best validation accuracy ',best_accuracy)

print('\n test accuracy using weights and biases of the highest validation accuracy: ',accuracy_test)

# cross validation ends






"""
comment the cross validation part and uncomment this for quick debugging

a=NeuralNetwork(10,0.001,0.9)
b=a.fit(x_train,y_train)
y_predicted=a.evaluate(x_test)
accuracy=accuracy_score(y_test, y_predicted,normalize=True)
print(accuracy)
"""

60000 train samples
10000 test samples
K Fold Cross Validation no.:  1
epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9
epoch  10
validation accuracy  0.65975
training accuracy  0.6533958333333333
K Fold Cross Validation no.:  2
epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9
epoch  10
validation accuracy  0.5955833333333334
training accuracy  0.5949791666666666
K Fold Cross Validation no.:  3
epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9
epoch  10
validation accuracy  0.573
training accuracy  0.5777708333333333
K Fold Cross Validation no.:  4
epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9
epoch  10
validation accuracy  0.6534166666666666
training accuracy  0.6464791666666667
K Fold Cross Validation no.:  5
epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9
epoch  10
validation accuracy  0.54625
training accuracy  0.54552083

'\ncomment the cross validation part and uncomment this for quick debugging\n\na=NeuralNetwork(10,0.001,0.9)\nb=a.fit(x_train,y_train)\ny_predicted=a.evaluate(x_test)\naccuracy=accuracy_score(y_test, y_predicted,normalize=True)\nprint(accuracy)\n'

Problem #2.1 (40 points): Implement a Convolutional Neural Network from scratch. Similarly to problem 1.1, we will be implementing the same architecture as the one shown in [Keras' CNN documentation](https://keras.io/examples/mnist_cnn/). That is:

- Input layer
- Convolutional hidden layer with 32 neurons, a kernel size of (3,3), and relu activation function
- Convolutional hidden layer with 64 neurons, a kernel size of (3,3), and relu activation function
- Maxpooling with a pool size of (2,2)
- Dropout with a value of 0.25
- Flatten layer
- Dense hidden layer, with 128 neurons, and relu activation function
- Dropout with a value of 0.5
- Output layer, using softmax as the activation function

Our loss function is categorical crossentropy and the evaluation will be done using accuracy, as in Problem 1.1. However, we will not be using the gradient optimizer known as Adadelta.

In [0]:
class ConvolutionalNeuralNetwork(object):
  def __init__(epochs, learning_rate):
    pass
  
  def fit(self):
    pass
  
  def evaluate(self):
    pass

Problem #2.2 (10 points): Train your convolutional neural network on the Fashion-MNIST dataset using 5-fold cross validation. Report accuracy on the folds, as well as on the test set.

In [0]:
import keras
from keras.datasets import fashion_mnist

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)